In [ ]:
import gymnasium as gym
import panda_gym
from sb3_contrib import TQC
from huggingface_sb3 import load_from_hub
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
import time
import numpy as np
from panda_gym.utils import distance
import random

In [ ]:
chk = load_from_hub(repo_id="BanUrsus/tqc-PandaPickAndPlace-v3", filename="tqc-PandaPickAndPlace-v3.zip")
stats = load_from_hub(repo_id="BanUrsus/tqc-PandaPickAndPlace-v3", filename="vec_normalize.pkl")
   
env = gym.make('PandaPickAndPlace-v3', render_mode="human", autoreset=False)

sim = env.sim
robot = env.robot
task = env.task

env = DummyVecEnv([lambda: env])
env = VecNormalize.load(stats, env)
env.training = False
env.norm_reward = False

In [ ]:
model = TQC.load(chk,env) 
env = model.env
obs = env.reset()

q1 = np.array([1.1 / 4 - 0.3, 0.7 / 4 , 0.4])
q2 = np.array([1.1 / 4 - 0.3, 3 * (0.7 / 4), 0.4])
q3 = np.array([(1.1 / 4) * 3 - 0.3, 0.7 / 4, 0.4])
q4 = np.array([(1.1 / 4) * 3 - 0.3, 3 * (0.7 / 4), 0.4])

random_states = []
solved_states = []


# predicates method / helper func
def calculate_proximity(ee_position, block_position):
    # Euclidean distance
    return np.linalg.norm(np.array(ee_position) - np.array(block_position))

def is_gripper_open(finger_width):
    return finger_width > 0.02

def is_gripper_correctly_oriented():
    return True  # NOTE: todo figure out how to do this correctly

def is_block_grasped(finger_width, proximity):
    return finger_width <= 0.04 and finger_width >= 0.02 and proximity < 0.05  

while len(random_states) < 10000 or len(solved_states) < 1000:
    action, _states = model.predict(obs, deterministic=True)
    # print(action)
    obs, reward, dones, info = env.step(action)

    # gather data
    ee = robot.get_ee_position()
    fw = robot.get_fingers_width()
    block = task.get_achieved_goal()
    goal = np.array(sim.get_base_position("target"))

    # predicates
    distance_to_block = calculate_proximity(ee, block)
    gripper_open = is_gripper_open(fw)
    gripper_correctly_oriented = is_gripper_correctly_oriented()
    block_grasped = is_block_grasped(fw, distance_to_block)

    # abstract state
    state = {
        "effector_pos": ee,
        "finger_width": fw,
        "gripper_open": gripper_open,
        "gripper_orientation_correct": gripper_correctly_oriented,
        "block_grasped": block_grasped,
        "d_q1": distance(ee, q1),
        "d_q2": distance(ee, q2),
        "d_q3": distance(ee, q3),
        "d_q4": distance(ee, q4),
        "distance_to_block": distance_to_block,
        "block_to_goal": distance(block, goal),
        "is_grasping": fw > 0.02 and fw < 0.04,
        "is_closed": fw < 0.01,
        "finished": dones[0]
    }

    # check matching w/ screen
    # current state output of abstract states
    print("Current State:")
    for key, value in state.items():
        print(f"{key}: {value}")

    time.sleep(10.5)

    if dones[0]:
        solved_states.append(state)
        obs = env.reset()
    else:
        if random.randint(0, 10) == 0:
            random_states.append(state)

print(f"finished: {len(random_states)} random states, {len(solved_states)} solved states")

In [ ]:
# robot.get_ee_position() gets us the position of the robots "fingers"
# robot.get_fingers_width() gets us the width between the robots "fingers"

# the table has dimensions (1.1, 0.7, 0.4) w/ x_offset=-0.3
# can partition the table into 4 quadrants
# q1 = (1.1 / 4 - 0.3, 0.7 / 4 , 0.4)
# q2 = (1.1 / 4 - 0.3, 3 * (0.7 / 4), 0.4)
# q3 = ((1.1 / 4) * 3 - 0.3, 0.7 / 4, 0.4)
# q4 = ((1.1 / 4) * 3 - 0.3, 3 * (0.7 / 4), 0.4)

# distance_to_q{} = d(robot.get_ee_position(), q{})

# grasping might be distance from effectors to block & a check to see if they arent completely open or closed
# distance_to_block = d(robot.get_ee_position(), desired_goal)


In [ ]:
# task.get_achieved_goal() where the block is now
# desired_goal = np.array(sim.get_base_position("target"))
# d = distance(achieved_goal, desired_goal)

In [ ]:
# looks like overall we keep the env in the wrapped form and just save the variables
# so we can mess with them elsewhere